In [ ]:
###########
### Inspired by 
# Python for Algorithmic Trading
# (c) Dr. Yves J. Hilpisch
# The Python Quants GmbH

##### Chris Greene 09.19.2017

import q    #
#import v20
import zmq  #import socket package
import numpy as np
import pandas as pd
import datetime as dt
import configparser

from urllib2 import Request, urlopen
import json
from pandas.io.json import json_normalize
import time
import requests
import hashlib
import hmac

####github for bittres api
#from bittrex import Bittrex
import websocket

##May not need this because well use the windows putty for now
# loading credentials
config = configparser.ConfigParser()
config.read('pyalgo.cfg')# need to create 

# setting up socket server
context = zmq.Context()
socket = context.socket(zmq.PUB)
socket.bind('tcp://0.0.0.0:5555')



class SMATrader(object):
    def __init__(self,SMA1, SMA2,*args, **kwargs):
        self.ticks = 0
        self.position = pd.DataFrame()
        self.data = pd.DataFrame()
        #self.instrument = instrument
        self.interval = '10s'
        self.SMA1 = SMA1
        self.SMA2 = SMA2
        self.units = pd.DataFrame()
        self.var1 = ''
        self.var2 = ''
        self.var3 = ''
        self.var4 = ''
        self.API_KEY = 'my-api-key' #config['Bittrex']['key']
        self.API_SECRET_KEY = b'my-secret-key' #config['Bittrex']['secret_key']

    def start(self):
        #prompt user to input the currency pairs
        _var1, _var2, _var3, _var4 = raw_input("Enter four currency pairs. Seperated  by a space: ").split()
        self.var1 = _var1
        self.var2 = _var2
        self.var3 = _var3
        self.var4 = _var4
        
        #ping server every 5sec for new data
        #need to check this logic
        ws = websocket.WebSocketApp('wss://bittrex.com/api/v1.1/public/getmarketsummaries', on_message=start_trader)  
        ws.run_forever(ping_interval=5) 
        
    def start_trader(self, ws, message):

        #this works fine, but may be other ways that are more straight forward
        #get all market summaries from Bittrex API
        #market_df = request_to_df('https://bittrex.com/api/v1.1/public/getmarketsummaries')
        market_todf = request_to_df(message)
        
        #pull only rows corresponding to the identified markets
        market_df = market_todf[market_todf['MarketName'].isin([self.var1, self.var2, self.var3, self.var4])]
        
        #add columns to dataframne
        market_df['mid'] =[]
        market_df['SMA1']=[]
        market_df['SMA2']=[]
        market_df['position']=[]
    
        #creates time index for dataframe
        #time_now= time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time()))
        timestamp = dt.datetime.now()
        
        # bring in new data, find currency pair, pull row info, pull neccesary columns, reinex with current date
        df_1row = market_df.loc[market_df.MarketName == self.var1]
        df_var1 = df_1row[['Created','MarketName','Ask','Bid','BaseVolume','mid','SMA1','SMA2', 'position','Last']]
        df_var1.index = [timestamp]
        #print df_var1
        
        #creating var2 row dataframe
        df_2row = market_df.loc[market_df.MarketName == self.var2]
        df_var2 = df_2row[['Created','MarketName','Ask','Bid','BaseVolume','mid','SMA1','SMA2', 'position','Last']]
        df_var2.index = [timestamp]
        #print df_var2
        
        #creating var3 row dataframe
        df_3row = market_df.loc[market_df.MarketName == self.var3]
        df_var3 = df_3row[['Created','MarketName','Ask','Bid','BaseVolume','mid','SMA1','SMA2', 'position','Last']]
        df_var3.index = [timestamp]
        #print d1f_var3
        
        #creating var4 row dataframe
        df_4row = market_df.loc[market_df.MarketName == self.var4]
        df_var4 = df_4row[['Created','MarketName','Ask','Bid','BaseVolume','mid','SMA1','SMA2', 'position','Last']]
        df_var4.index = [timestamp]
        #print df_var4

        #combine data frames into a list
        frames = [df_var1, df_var2, df_var3, df_var4]
        
        #concat data frames into a multiIndex dataframe
        df_result = pd.concat(frames, keys=[self.var1, self.var2, self.var3, self.var4 ],axis=1)
            
        #run function to determine buy/sell signal to create order
        on_success(df_result)
        
        #checks if an abritrary tick limit has been reached and cancels all orders 
        #by position close

        # logic to close out on all open orders 
        df_open_orders=pd.DataFrame()
        
        if self.ticks == 500:
            list_resam = list(df_result.columns.levels[0])
            for mrkt in list_resam:
                #buy_cur = mrkt.split('-',1)[0]
                #sell_cur= mrkt.split('-',1)[1] 
                
                #get all open orders
                df_open_orders = get_open_orders(mrkt)
                
                #df_uuid_col = df_open_orders['Order_Unid'][]
                
                #close all open orders
                for order in df_open_orders['Order_Unid']:
                    close_orders(order)     
    
    
    #convert api request from json to dataframe
    def request_to_df(request):
        response = urlopen(request)
        elevations = response.read()
        data = json.loads(elevations)
        df= json_normalize(data['result'])
        return df
    
    #create limit order via bittrex api
    def create_order(self, market, units, rate):
        url = 'https://bittrex.com/api/v1.1/market/buylimit?apikey=' + self.API_KEY + 
        '&market=' + market + '&quantity=' + str(units) + '&rate=' + format_float(rate)
        # order data
        socket.send_string('SMA\n' + str(url))
        print('\n', str(url))
        q(str(url))
        # market data
        msg = 'SMA | Market data:\n%s' % self.resam.iloc[-1]
        socket.send_string(msg)
        print('\n', msg)
        q(msg)
        #double check if this is the correct logic
        return signed_request(url)
    
    #close orders via api
    def close_orders(order_uuid):
        if len(order_uuid)>1:
            url = 'https://bittrex.com/api/v1.1/market/cancel?apikey=' + API_KEY + '&uuid=' + order_uuid  
            request = signed_request(url)
            return request_to_df(request)
    
    #get all open orders
    def get_open_orders(market):
        url = 'https://bittrex.com/api/v1.1/market/getopenorders?apikey=' + API_KEY + '&market=' + market
        request = signed_request(url)
        return request_to_df(request)
    
    #sign request with private key
    def signed_request(url):
        now = time.time()
        url += '&nonce=' + str(now)
        signed = hmac.new(self.API_SECRET_KEY, url.encode('utf-8'), hashlib.sha512).hexdigest()
        headers = {'apisign': signed}
        r = requests.get(url, headers=headers)
    return r.json()
   
    #get balance of specified market
    def get_balance(currency):
        url = 'https://bittrex.com/api/v1.1/account/getbalance?apikey=' + API_KEY + '&currency=' + currency
        res = signed_request(url)
        #res_df_x = pd.DataFrame(res)
        #res_df= res_df_x['result']
        #print(res_df)
        if res['result']['Available'] is not None and len(res['result']) > 0:
            return res['result']['Available']
        else:
            return 0

    #logic to append single instance of ticker data,
    #then calculate buy or sell signals, then execute trade
    def on_success(self, df_msg):
        #tally current tick
        self.ticks += 1
        msg = 'SMA | %s | retrieved new data | %5d' % (str(dt.datetime.now()),
                                                       self.ticks)
        #record ticker timestamp
        socket.send_string(msg)
        print(msg)
        
        #append ticker instance to ticker dataframe
        self.data = self.data.append(df_msg)
        
        #resample data into 10sec interval, aggregates metrics by 10sec intervals
        self.resam = self.data.resample(self.interval).last().ffill()
        
        #get column names on the multiIndex dataframe
        list_resam = list(self.resam.columns.levels[0])
        
        #initialize selling & buying currency dataframes to keep track of balances
        buy_cur=pd.DataFrame()
        buy_cur.columns = list_resam
        
        sell_cur=pd.DataFrame()
        sell_cur.columns= list_resam
        
        cur_start_bal=pd.DataFrame()
        cur_start_bal.columns = list_resam
        
        #capture starting balances
        if self.ticks == 1:
            for m in list_resam:
                cur_start_bal[m] = get_balance(m)
        
        #calculate signal for each market and execute a buy/sell trade
        for mrkt in list_resam:
            
            #extract the buy/sell currency from market name
            buy_cur[mrkt] = mrkt.split('-',1)[0]
            sell_cur[mrkt] = mrkt.split('-',1)[1]
            
            #checks if there is enough data points to do the smoothing average calc
            if len(self.resam[mrkt]) > self.SMA2:
                
                #calculates the mid price of the order book
                self.resam[mrkt]['mid'] = (self.resam[mrkt]['bid'] +
                                     self.resam[mrkt]['ask']) / 2
                
                #calculates the rolling avg for SMA1
                self.resam[mrkt]['SMA1'] = self.resam[mrkt]['mid'].rolling(self.SMA1).mean()
                
                #calculates the rolling avg for SMA2
                self.resam[mrkt]['SMA2'] = self.resam[mrkt]['mid'].rolling(self.SMA2).mean()
                
                #calculates buy or sell signal
                self.resam[mrkt]['position'] = np.where(self.resam[mrkt]['SMA1'] >
                                                  self.resam[mrkt]['SMA2'], 1, -1)
                # print(self.resam[['bid', 'ask', 'SMA1', 'SMA2',
                #                   'position']].tail())
                    
                    #get balance for buy/sell currencies
                    cur_balance_buy = get_balance(buy_cur[mrkt])
                    cur_balance_sell=get_balance(sell_cur[mrkt])
                    
                    #check signal and confirm enough balance to make trade
                    if self.resam[mrkt]['position'].ix[-1] == 1 and 1-(cur_start_bal[mrkt]['Balance']-
                                                cur_balance_buy['Balance'])/cur_start_bal[mrkt]['Balance'] > .09999:
                        #determine previous signal state
                        if self.position[mrkt] == 0 :
                            self.create_order(mrkt,cur_start_bal[mrkt]['Balance']/20,self.resam[mrkt]['mid'])
                        elif self.position[mrkt] == -1:
                            self.create_order(mrkt,cur_start_bal[mrkt]['Balance']/20 * 2,self.resam[mrkt]['mid'])
                        self.position[mrkt] = 1
                    
                    #check for negative signal & balance over start bal * .05
                    elif self.resam[mrkt]['position'].ix[-1] == -1 and 
                                                          cur_balance_sell['Balance'] > cur_start_bal[mrkt]['Balance']/20:
                        #determine previous signal state
                        if self.position[mrkt] == 0:
                            self.create_order(mrkt,-cur_start_bal[mrkt]['Balance']/20,self.resam[mrkt]['mid'])
                        elif self.position[mrkt] == 1:
                            self.create_order(mrkt,-cur_start_bal[mrkt]['Balance']/20 * 2,self.resam[mrkt]['mid'])
                        self.position[mrkt] = -1
                    
                    #check for negative signal & balance over current bal < start bal* .05
                    elif self.resam[mrkt]['position'].ix[-1] == -1 and 
                                    cur_balance_sell['Balance'] < (cur_start_bal[mrkt]['Balance']/20) + 0.001:
                        #determine previous signal state
                        if self.position[mrkt] == 0:
                            self.create_order(mrkt,-cur_start_bal[mrkt]['Balance'],self.resam[mrkt]['mid'])
                        elif self.position[mrkt] == 1:
                            self.create_order(mrkt,-cur_start_bal[mrkt]['Balance'] * 2,self.resam[mrkt]['mid'])
                        self.position[mrkt] = -1
    
    def on_error(self):
        ''' Call this method in case of any error/exception. '''
        socket.send_string('A serious error occured.')
        print('\nA serious error occured.')
        q('A serious error occured.')
        # initiate any required/desired risk mitigation task

if __name__ == '__main__':
    sma = SMATrader(SMA1=5, SMA2=10)
    sma.start()